Code adapted from: https://menvuthy.github.io/Vuthy/forest%20cover/forest-cover-cambo-copy/ + https://blog.gishub.org/earth-engine-tutorial-30-how-to-get-image-properties-and-descriptive-statistics + https://developers.google.com/earth-engine/tutorials/tutorial_forest_03a

#### Further Resources
- https://github.com/RSPB/GFCalculator/blob/master/GFCalculator.py

In [1]:
import ee
import geemap
from geemap import *
import json
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import os
# !pip install geemap


In [2]:
Map = geemap.Map()


### Set Region Of Interest

In [5]:
#clip image to only include aoi region
file_path = os.path.abspath('/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/Data/GeoJSONS/PIREDD_mai_ndombe.geojson')
with open(file_path) as f:
    studyRegion = json.load(f)

studyRegion = ee.FeatureCollection(studyRegion).first().geometry()
#print(studyRegion)
Map.setCenter(18.4276, -2.6357,7)
Map.addLayer(studyRegion,{},"studyRegion")
Map

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Element.geometry",
    "arguments": {
      "feature": {
        "functionInvocationValue": {
          "functionName": "Collection.first",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection",
                "arguments": {
                  "features": {
                    "arrayValue": {
                      "values": [
                        {
                          "functionInvocationValue": {
                            "functionName": "Feature",
                            "arguments": {
                              "geometry": {
                                "functionInvocationValue": {
                                  "functionName": "GeometryConstructors.Polygon",
                                  "arguments": {
                                    "coordinates": {
                                      "constantValu

Map(center=[-2.6357, 18.4276], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButto…

### Process Satellite Data

In [7]:
point = ee.Geometry.Point(18.4276, -2.6357)

#define cloud free composite



#define cloud mask
def cloudMask(image):
  #Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    #Get the pixel QA band.
    qa = image.select('pixel_qa')
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)
    
#grab image collection for 2013
l8_2013 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2013-01-01', '2013-12-31')
cloud_free_image_2013 = l8_2013.map(cloudMask)#apply cloud mask
median_image_2013 = cloud_free_image_2013.median().select(['B1','B2','B3','B4','B5','B6','B7','B10','B11','pixel_qa'])#take median pixel values for all bands
l8_2013 = median_image_2013.clip(studyRegion)#clip to studyRegion

#grab image collection for 2019
l8_2019 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2013-01-01', '2013-12-31')
cloud_free_image_2019 = l8_2019.map(cloudMask)#apply cloud mask
median_image_2019 = cloud_free_image_2019.median().select(['B1','B2','B3','B4','B5','B6','B7','B10','B11','pixel_qa'])#take median pixel values for all bands
l8_2019 = median_image_2019.clip(studyRegion)#clip to studyRegion

vis_params_1 = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'] #RGB Composite
}

vis_params_2 = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B6', 'B7'] #RGB Composite
}

Map.addLayer(l8_2013, vis_params_1, "Landsat-8_2013")
Map.addLayer(l8_2019, vis_params_2, "Landsat-8_2019")
Map

Map(bottom=16924.0, center=[7, ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConst…

In [21]:
#print area - doesn't work, need to know how to translate this into python properly
#print('The total area is:', l8_2013.multiply(ee.Image.pixelArea()))

### Explore Hansen Data for Region

In [26]:
hansen_2020 = ee.Image('UMD/hansen/global_forest_change_2020_v1_8').clip(studyRegion)

#Accessing the forest loss and gain layers
#update yearly so get the most recent version
lossImage = hansen_2020.select(['loss'])
gainImage = hansen_2013.select(['gain'])

#merging the loss and gain layers
gainAndLoss = gainImage.And(lossImage)

#Accessing forest cover layer
treeCover = hansen_2013.select(['treecover2000'])

#Add the tree cover layer in green.
Map.addLayer(treeCover.updateMask(treeCover),
    {"palette": ['000000', '00FF00'], max: 100}, 'Forest Cover');

#Add the loss layer in red.
Map.addLayer(lossImage.updateMask(lossImage),
    {"palette": ['FF0000']}, 'Loss');

#Add the gain layer in blue.
Map.addLayer(gainImage.updateMask(gainImage),
    {"palette": ['0000FF']}, 'Gain');

#Add the merged gain and loss in purple
Map.addLayer(gainAndLoss.updateMask(gainAndLoss),
    {"palette": 'FF00FF'}, 'Gain and Loss')
Map

Map(bottom=134657.0, center=[-4.506607001752771, 16.28105163574219], controls=(WidgetControl(options=['positio…

### Chart Yearly Forest Loss

In [35]:
#calculate loss statistics
hansen_2020 = ee.Image('UMD/hansen/global_forest_change_2020_v1_8').clip(studyRegion)

#Accessing the forest loss layer
lossImage = hansen_2020.select(['loss'])
lossAreaImage = lossImage.multiply(ee.Image.pixelArea())
lossYear = hansen_2020.select(['lossyear'])
lossByYear = lossAreaImage.addBands(lossYear).reduceRegion(**{"reducer": ee.Reducer.sum().group(1),
                                                            "geometry": studyRegion,
                                                            "scale": 30,
                                                            "maxPixels":1e10})

lossByYear.getInfo()

{'groups': [{'group': 1, 'sum': 188105538.16541344},
  {'group': 2, 'sum': 166824510.77106336},
  {'group': 3, 'sum': 59643969.930230595},
  {'group': 4, 'sum': 49336846.5878746},
  {'group': 5, 'sum': 171941074.28023493},
  {'group': 6, 'sum': 82827402.9937208},
  {'group': 7, 'sum': 90890315.46636316},
  {'group': 8, 'sum': 160244300.74224204},
  {'group': 9, 'sum': 136387790.62678176},
  {'group': 10, 'sum': 267642026.16495433},
  {'group': 11, 'sum': 131746008.87300211},
  {'group': 12, 'sum': 167577984.96354285},
  {'group': 13, 'sum': 384472114.5348055},
  {'group': 14, 'sum': 437352112.82284284},
  {'group': 15, 'sum': 196947915.66200185},
  {'group': 16, 'sum': 347412787.4840143},
  {'group': 17, 'sum': 389135461.53551406},
  {'group': 18, 'sum': 479621949.6379995},
  {'group': 19, 'sum': 331371710.52089316},
  {'group': 20, 'sum': 407230537.1872507}]}

In [45]:
#conver stats into years (formatting)
def format(stats):
    d = ee.Dictionary(stats);
    return [ee.Number(d.get('group')).format("20%02d"), d.get('sum')]

stats = ee.List(lossByYear.get('groups'))

statsFormatted = format(stats)
statsDictionary = ee.Dictionary(statsFormatted.flatten)
statsFormatted.getInfo()

AttributeError: 'list' object has no attribute 'flatten'

In [ ]:
#get stats for the region of interest

yearly_hansen_array = []

lossImage = hansen_2013.select(['loss'])

stats = lossImage.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': studyRegion,
  'scale': 30,
  'maxPixels':1e10
})

stats.getInfo()


In [ ]:
#computation of yearly forest loss - not really sure here what the ouptu statistics are? it also has 
#various classes which it is not obvious what they are either
input_zone = ee.FeatureCollection(studyRegion)
out_path = ('/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/GEE_examples/Output Data/Hansen_Results')
forest_cover = os.path.join(out_path, 'forest-cover-2013.csv')  
geemap.zonal_statistics_by_group(forestAt2015.updateMask(forestAt2015), input_zone, forest_cover, statistics_type='SUM', denominator=10000, decimal_places=2)


In [ ]:
#create yearly forest cover map - will work on making it more efficient with a loop later on

loss = image.select(['loss'])
lossYear = hansen_2013.select(['lossyear'])
forest = hansen_2013.select(['treecover2000'])

# visualization setting
vis = {
'min': 0, 
'max': 100, 
'palette': ['#000000', '#005500', '#00AB00', '#00FF00']
}

#2013
lossInFirst13 = lossYear.gte(1).And(lossYear.lte(13))
forestAt2013 = forest.where(lossInFirst13.eq(1), 0)
#2014
lossInFirst14 = lossYear.gte(1).And(lossYear.lte(14))
forestAt2014 = forest.where(lossInFirst14.eq(1), 0)
#2015
lossInFirst15 = lossYear.gte(1).And(lossYear.lte(15))
forestAt2015 = forest.where(lossInFirst15.eq(1), 0)
#2016
lossInFirst16 = lossYear.gte(1).And(lossYear.lte(16))
forestAt2016 = forest.where(lossInFirst16.eq(1), 0)
#2017
lossInFirst17 = lossYear.gte(1).And(lossYear.lte(17))
forestAt2017 = forest.where(lossInFirst17.eq(1), 0)
#2018
lossInFirst18 = lossYear.gte(1).And(lossYear.lte(18))
forestAt2018 = forest.where(lossInFirst18.eq(1), 0)
#2019
lossInFirst19 = lossYear.gte(1).And(lossYear.lte(19))
forestAt2019 = forest.where(lossInFirst19.eq(1), 0)

Map.addLayer(forestAt2013.updateMask(forestAt2013), vis, 'Forest in 2001')
Map.addLayer(forestAt2019.updateMask(forestAt2019), vis, 'Forest in 2019')
Map